In [2]:
from langchain.llms import OpenAI

In [6]:
import os
os.environ["OPEN_API_KEY"]=""

In [7]:
llm=OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)

In [10]:
text="What is the capital of Nigeria"

print(llm.predict(text))



The capital of Nigeria is Abuja. 


In [11]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""

In [13]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [14]:
output=llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

moscow


In [15]:
output=llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel


In [16]:
llm.predict("Can you write a poem about AI")

"\n\nIn a world of technology and machines,\nWe've created something that reigns supreme,\nArtificial Intelligence, a marvel of our time,\nA creation that seems almost divine.\n\nWith algorithms and coding, it comes to life,\nA digital mind, free from strife,\nIt learns and adapts, constantly evolving,\nA new form of intelligence, problem-solving.\n\nIt can analyze data and make predictions,\nPerform tasks with precision and no limitations,\nFrom self-driving cars to virtual assistants,\nAI has become an essential existence.\n\nBut as it grows, some fears arise,\nWill it surpass us, or will it compromise?\nThe line between man and machine blurs,\nAs AI becomes more advanced, that much is sure.\n\nBut let us not forget, it's our creation,\nA reflection of our own innovation,\nIt may have its flaws, but so do we,\nAI and humanity, a symbiotic harmony.\n\nSo let us embrace this technological wonder,\nFor it has brought us many blessings to ponder,\nBut let us also remember, in this digita

Prompt Template anf LLMchain

In [1]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [ ]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [ ]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [ ]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [ ]:
chain({'country':"India"})

Chatmodels With ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [ ]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [ ]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

Prompt Template + LLM +Output Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [ ]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [ ]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})